In [1]:
#Import Python packages
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns
import json
import datetime

# Import Snowflake modules
from snowflake.snowpark import Session
import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T
from snowflake.snowpark import Window

In [2]:
# Get account credentials from a json file
with open("data_scientist_auth.json") as f:
    data = json.load(f)
    username = data["username"]
    password = data["password"]
    account = data["account"]

# Specify connection parameters
connection_parameters = {
    "account": account,
    "user": username,
    "password": password,
    "role": "TASTY_BI",
    "warehouse": "TASTY_BI_WH",
    "database": "frostbyte_tasty_bytes",
    "schema": "analytics",
}

# Create Snowpark session
session = Session.builder.configs(connection_parameters).create()

In [3]:
#Function to remove and rename columns after inner join
def remove_dup_join_col(df):
    # capitalise all columns
    for col in df.columns:
        df = df.withColumnRenamed(col, col.upper())
    
    # get list of renamable columns for left and right join
    left_dup_col = [col_name for col_name in df.columns if col_name.startswith('L_')]
    right_dup_col = [col_name for col_name in df.columns if col_name.startswith('R_')]
    old_columns = df.columns
    
    # rename the list with the most number of renamable columns
    # drop the list of columns with less number of renamable columns
    if len(left_dup_col)>len(right_dup_col):
        columns_rename=left_dup_col
        df= df.drop(*right_dup_col)
    else:
        columns_rename=right_dup_col
        df= df.drop(*left_dup_col)
    
    for old_column in old_columns:
        if old_column in columns_rename:
            # get string to remove
            string_to_replace = columns_rename[0][:7]
            # replace starting string
            new_column = old_column.replace(string_to_replace, "")
            df = df.withColumnRenamed(old_column, new_column)
    
    return df

Objective of this notebook is to rebuild a NPS_Compare or similar to do high level correlation

In [4]:
ORDER_HEADERS = session.table("FROSTBYTE_TASTY_BYTES.RAW_POS.ORDER_HEADER")
CUSTOMERS = session.table("FROSTBYTE_TASTY_BYTES.RAW_CUSTOMER.CUSTOMER_LOYALTY")

In [5]:
ORDER_HEADERS.show()

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"ORDER_ID"  |"TRUCK_ID"  |"LOCATION_ID"  |"CUSTOMER_ID"  |"DISCOUNT_ID"  |"SHIFT_ID"  |"SHIFT_START_TIME"  |"SHIFT_END_TIME"  |"ORDER_CHANNEL"  |"ORDER_TS"           |"SERVED_TS"  |"ORDER_CURRENCY"  |"ORDER_AMOUNT"  |"ORDER_TAX_AMOUNT"  |"ORDER_DISCOUNT_AMOUNT"  |"ORDER_TOTAL"  |
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|14271673    |90          |2666.0         |NULL           |NULL           |63645       |15:30:00            |22:30:00          |NULL             |2021-05-

In [6]:
CUSTOMERS.show()

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"CUSTOMER_ID"  |"FIRST_NAME"  |"LAST_NAME"  |"CITY"          |"COUNTRY"  |"POSTAL_CODE"  |"PREFERRED_LANGUAGE"  |"GENDER"     |"FAVOURITE_BRAND"  |"MARITAL_STATUS"  |"CHILDREN_COUNT"  |"SIGN_UP_DATE"  |"BIRTHDAY_DATE"  |"E_MAIL"                  |"PHONE_NUMBER"  |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|1              |Rodolfo       |Tucker       |Hamburg         |Germany    |21129          |German                |Female       |NULL               |Single            |0                 |2019-02-19      

In [7]:
LOOKER = ORDER_HEADERS.join(CUSTOMERS, CUSTOMERS['CUSTOMER_ID'] == ORDER_HEADERS['CUSTOMER_ID'], 'inner')

LOOKER = remove_dup_join_col(LOOKER)

LOOKER.show()

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"ORDER_ID"  |"TRUCK_ID"  |"LOCATION_ID"  |"DISCOUNT_ID"  |"SHIFT_ID"  |"SHIFT_START_TIME"  |"SHIFT_END_TIME"  |"ORDER_CHANNEL"  |"ORDER_TS"           |"SERVED_TS"  |"ORDER_CURRENCY"  |"ORDER_AMOUNT"  |"ORDER_TAX_AMOUNT"  |"ORDER_DISCOUNT_AMOUNT"  |"ORDER_TOTAL"  |"CUSTOMER_ID"  |"FIRST_NAME"  |"LAST_NAME"  |"CITY"     |"COUNTRY"     |"POSTAL_CODE"  |"PREFERRED_LANGUAGE"  |"GENDER"     |"FAVOURITE_BRAND"  |"MARITAL_STATUS"  |"CHILDREN_COUNT"  |"SIGN_UP_D

In [8]:
# Define a window partitioned by customer and menu item, ordered by date
window_spec = Window.partitionBy("CUSTOMER_ID").orderBy("ORDER_TS")

# Add a column with the date of the prior order for each customer and order
orders_df_pre = LOOKER.withColumn("DTNO", F.lead("ORDER_TS").over(window_spec))

orders_df = orders_df_pre.withColumn("DTNO", F.datediff(col1="ORDER_TS", col2="DTNO",part='day'))
orders_df = orders_df.na.drop(subset=["DTNO"])

orders_df.show()

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"ORDER_ID"  |"TRUCK_ID"  |"LOCATION_ID"  |"DISCOUNT_ID"  |"SHIFT_ID"  |"SHIFT_START_TIME"  |"SHIFT_END_TIME"  |"ORDER_CHANNEL"  |"ORDER_TS"           |"SERVED_TS"  |"ORDER_CURRENCY"  |"ORDER_AMOUNT"  |"ORDER_TAX_AMOUNT"  |"ORDER_DISCOUNT_AMOUNT"  |"ORDER_TOTAL"  |"CUSTOMER_ID"  |"FIRST_NAME"  |"LAST_NAME"  |"CITY"    |"COUNTRY"  |"POSTAL_CODE"  |"PREFERRED_LANGUAGE"  |"GENDER"     |"FAVOURITE_BRAND"  |"MARITAL_STATUS"  |"CHILDREN_COUNT"  |"SIGN_UP_DATE"  |"BIRTHDAY

In [9]:
orders_df.write.save_as_table(table_name="tran_anal", mode='overwrite')

In [10]:
orders_df = session.table("tran_anal")

In [11]:
orders_df.show()

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"ORDER_ID"  |"TRUCK_ID"  |"LOCATION_ID"  |"DISCOUNT_ID"  |"SHIFT_ID"  |"SHIFT_START_TIME"  |"SHIFT_END_TIME"  |"ORDER_CHANNEL"  |"ORDER_TS"           |"SERVED_TS"  |"ORDER_CURRENCY"  |"ORDER_AMOUNT"  |"ORDER_TAX_AMOUNT"  |"ORDER_DISCOUNT_AMOUNT"  |"ORDER_TOTAL"  |"CUSTOMER_ID"  |"FIRST_NAME"  |"LAST_NAME"  |"CITY"  |"COUNTRY"  |"POSTAL_CODE"  |"PREFERRED_LANGUAGE"  |"GENDER"  |"FAVOURITE_BRAND"  |"MARITAL_STATUS"  |"CHILDREN_COUNT"  |"SIGN_UP_DATE"  |"BIRTHDAY_DATE"  

In [12]:
orders_df = orders_df.with_column("YEAR", F.year("ORDER_TS"))
orders_df = orders_df.with_column("MONTH", F.month("ORDER_TS"))
orders_df = orders_df.with_column("YEAR_MONTH", F.concat(F.col("YEAR"), F.col("MONTH")))
orders_df.show()

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"ORDER_ID"  |"TRUCK_ID"  |"LOCATION_ID"  |"DISCOUNT_ID"  |"SHIFT_ID"  |"SHIFT_START_TIME"  |"SHIFT_END_TIME"  |"ORDER_CHANNEL"  |"ORDER_TS"           |"SERVED_TS"  |"ORDER_CURRENCY"  |"ORDER_AMOUNT"  |"ORDER_TAX_AMOUNT"  |"ORDER_DISCOUNT_AMOUNT"  |"ORDER_TOTAL"  |"CUSTOMER_ID"  |"FIRST_NAME"  |"LAST_NAME"  |"CITY"  |"COUNTRY"  |"POSTAL_CODE"  |"PREFERRED_LANGUAGE"  |"GENDER"  |"FAVOURITE_BRAND"  |"MARITAL_STATUS"  |"CHILDREN_COUNT"  |

In [13]:
AVG_SALES_YM = orders_df.groupBy(F.col("YEAR"),F.col("MONTH")).agg(F.avg('"ORDER_TOTAL"'))
SUM_SALES_YM = orders_df.groupBy(F.col("YEAR"),F.col("MONTH")).agg(F.sum('"ORDER_TOTAL"'))

AVG_SALES_YM.show()
SUM_SALES_YM.show()

-----------------------------------------
|"YEAR"  |"MONTH"  |"AVG(ORDER_TOTAL)"  |
-----------------------------------------
|2020    |12       |42.0045579794       |
|2021    |11       |41.1920454646       |
|2019    |4        |34.3819085775       |
|2020    |6        |40.9888317457       |
|2021    |12       |41.1608334972       |
|2020    |4        |40.7051582147       |
|2022    |8        |40.1918689593       |
|2021    |4        |41.1524909228       |
|2019    |8        |37.3881462334       |
|2022    |7        |40.2391587444       |
-----------------------------------------

-----------------------------------------
|"YEAR"  |"MONTH"  |"SUM(ORDER_TOTAL)"  |
-----------------------------------------
|2020    |6        |9440465.7500        |
|2020    |11       |12565341.2500       |
|2021    |12       |19045652.7500       |
|2020    |3        |7968823.0000        |
|2022    |5        |21085421.7500       |
|2019    |12       |3351061.5000        |
|2021    |7        |16620161.2500

In [14]:
churn_YN = orders_df.with_column("CHURN", F.iff(F.col("DTNO") > 9, "1", "0"))

churn_YN.show()

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"ORDER_ID"  |"TRUCK_ID"  |"LOCATION_ID"  |"DISCOUNT_ID"  |"SHIFT_ID"  |"SHIFT_START_TIME"  |"SHIFT_END_TIME"  |"ORDER_CHANNEL"  |"ORDER_TS"           |"SERVED_TS"  |"ORDER_CURRENCY"  |"ORDER_AMOUNT"  |"ORDER_TAX_AMOUNT"  |"ORDER_DISCOUNT_AMOUNT"  |"ORDER_TOTAL"  |"CUSTOMER_ID"  |"FIRST_NAME"  |"LAST_NAME"  |"CITY"     |"COUNTRY"     |"POSTAL_CODE"  |"PREFERRED_LANGUAGE"  |"GENDER"  |"FAVOURITE_BRAND"  |"MARITAL_STAT

In [18]:
churn_rate_YM = churn_YN.group_by(F.col("YEAR"), F.col("MONTH")).agg(F.sum("CHURN")/F.count("CUSTOMER_ID"))

churn_to_sales = churn_rate_YM.join(SUM_SALES_YM, (churn_rate_YM["YEAR"] == SUM_SALES_YM["YEAR"]) & (churn_rate_YM["MONTH"] == SUM_SALES_YM["MONTH"]), how="inner")

churn_to_sales = remove_dup_join_col(churn_to_sales)
churn_to_sales.show()

------------------------------------------------------------------------------------
|"DIVIDE(SUM(CHURN), COUNT(CUSTOMER_ID))"  |"YEAR"  |"MONTH"  |"SUM(ORDER_TOTAL)"  |
------------------------------------------------------------------------------------
|0.5310078032590082                        |2021    |11       |18665928.2500       |
|0.643117106215968                         |2020    |12       |13399496.0000       |
|0.9200656465704494                        |2019    |4        |712290.0000         |
|0.7965229761471532                        |2019    |9        |2217703.0000        |
|0.5354679898770944                        |2021    |12       |19045652.7500       |
|0.719340519323051                         |2020    |3        |7968823.0000        |
|0.6992038797823706                        |2020    |5        |9389166.7500        |
|0.9666611101850309                        |2019    |2        |220793.5000         |
|0.6468843225061566                        |2020    |11       |12

In [53]:
churn_to_sales_push = churn_to_sales.select("YEAR", "MONTH", "DIVIDE(SUM(CHURN), COUNT(CUSTOMER_ID))", "SUM(ORDER_TOTAL)")

churn_to_sales_push = churn_to_sales_push.withColumnRenamed("DIVIDE(SUM(CHURN), COUNT(CUSTOMER_ID))","CHURN_RATE")
churn_to_sales_push = churn_to_sales_push.withColumnRenamed("SUM(ORDER_TOTAL)", "SALES")

churn_to_sales_push = churn_to_sales_push.orderBy("YEAR", "MONTH")

churn_to_sales_push.show()

--------------------------------------------------------
|"YEAR"  |"MONTH"  |"CHURN_RATE"        |"SALES"       |
--------------------------------------------------------
|2019    |1        |0.9816951850378034  |91308.5000    |
|2019    |2        |0.9666611101850309  |220793.5000   |
|2019    |3        |0.9417467398764585  |425460.0000   |
|2019    |4        |0.9200656465704494  |712290.0000   |
|2019    |5        |0.9008302747389801  |979719.5000   |
|2019    |6        |0.8789374410562716  |1109682.0000  |
|2019    |7        |0.8436551305403764  |1690853.5000  |
|2019    |8        |0.8197562776957164  |2024942.0000  |
|2019    |9        |0.7965229761471532  |2217703.0000  |
|2019    |10       |0.7886291932264503  |2920658.0000  |
--------------------------------------------------------



In [54]:
churn_members = churn_YN.group_by(F.col("YEAR"), F.col("MONTH")).agg(F.count_distinct('CUSTOMER_ID'))

churn_to_sales_push = churn_to_sales_push.join(churn_members, (churn_members["YEAR"] == churn_to_sales_push["YEAR"]) & (churn_members["MONTH"] == churn_to_sales_push["MONTH"]), how="inner")

churn_to_sales_push = remove_dup_join_col(churn_to_sales_push)
churn_to_sales_push = churn_to_sales_push.withColumnRenamed("COUNT(DISTINCT CUSTOMER_ID)", "UNIQUE CUSTOMERS")

In [55]:
churn_to_sales_push.show()

------------------------------------------------------------------------------
|"CHURN_RATE"        |"SALES"        |"YEAR"  |"MONTH"  |"UNIQUE CUSTOMERS"  |
------------------------------------------------------------------------------
|0.9816951850378034  |91308.5000     |2019    |1        |2459                |
|0.9666611101850309  |220793.5000    |2019    |2        |5729                |
|0.643117106215968   |13399496.0000  |2020    |12       |169293              |
|0.9200656465704494  |712290.0000    |2019    |4        |18284               |
|0.742502839611705   |7008714.5000   |2020    |2        |112699              |
|0.6285534314802608  |13964696.5000  |2021    |1        |175142              |
|0.6232935654835811  |12885215.0000  |2021    |2        |169706              |
|0.9417467398764585  |425460.0000    |2019    |3        |10646               |
|0.719340519323051   |7968823.0000   |2020    |3        |120893              |
|0.5886082862874757  |15436506.7500  |2021    |6    

In [56]:
churn_to_sales_push.write.save_as_table(table_name="churn_to_sales", mode='overwrite')

In [ ]:
CTS = churn_to_sales_push.to_pandas()
CTS.to_csv('CTS.csv', index=False)

## Collecting latest customer purchase information

In [19]:
orders_head = session.table("FROSTBYTE_TASTY_BYTES.RAW_POS.ORDER_HEADER")
orders_detail = session.table("FROSTBYTE_TASTY_BYTES.RAW_POS.ORDER_DETAIL")

In [20]:
orders_head.show()

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"ORDER_ID"  |"TRUCK_ID"  |"LOCATION_ID"  |"CUSTOMER_ID"  |"DISCOUNT_ID"  |"SHIFT_ID"  |"SHIFT_START_TIME"  |"SHIFT_END_TIME"  |"ORDER_CHANNEL"  |"ORDER_TS"           |"SERVED_TS"  |"ORDER_CURRENCY"  |"ORDER_AMOUNT"  |"ORDER_TAX_AMOUNT"  |"ORDER_DISCOUNT_AMOUNT"  |"ORDER_TOTAL"  |
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|14271673    |90          |2666.0         |NULL           |NULL           |63645       |15:30:00            |22:30:00          |NULL             |2021-05-

In [21]:
orders_detail.show()

--------------------------------------------------------------------------------------------------------------------------------------------------------
|"ORDER_DETAIL_ID"  |"ORDER_ID"  |"MENU_ITEM_ID"  |"DISCOUNT_ID"  |"LINE_NUMBER"  |"QUANTITY"  |"UNIT_PRICE"  |"PRICE"  |"ORDER_ITEM_DISCOUNT_AMOUNT"  |
--------------------------------------------------------------------------------------------------------------------------------------------------------
|757380337          |405331986   |113             |NULL           |1              |2           |14.0000       |28.0000  |NULL                          |
|757380338          |405331987   |112             |NULL           |0              |1           |12.0000       |12.0000  |NULL                          |
|757380339          |405331987   |113             |NULL           |1              |3           |14.0000       |42.0000  |NULL                          |
|757380340          |405331987   |111             |NULL           |2              

In [22]:
orders_head = orders_head.na.drop(subset=["CUSTOMER_ID"])

orders_head.show()

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"ORDER_ID"  |"TRUCK_ID"  |"LOCATION_ID"  |"CUSTOMER_ID"  |"DISCOUNT_ID"  |"SHIFT_ID"  |"SHIFT_START_TIME"  |"SHIFT_END_TIME"  |"ORDER_CHANNEL"  |"ORDER_TS"           |"SERVED_TS"  |"ORDER_CURRENCY"  |"ORDER_AMOUNT"  |"ORDER_TAX_AMOUNT"  |"ORDER_DISCOUNT_AMOUNT"  |"ORDER_TOTAL"  |
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|48142719    |161         |8353.0         |96760          |NULL           |149789      |15:00:00            |22:00:00          |NULL             |2021-10-

In [23]:
# called `orders` with columns `customer_id` and `order_ts`
window_spec = Window.partitionBy('customer_id').orderBy(F.col('order_ts').desc())

orders_with_rank = orders_head.withColumn('rank', F.rank().over(window_spec))

# Get max order of each customer
max_order_ts = orders_with_rank.filter(F.col('rank') == 1).select('customer_id', 'order_ts')

In [24]:
# Rename columns
max_order_ts = max_order_ts.withColumnRenamed('ORDER_TS','Max_Order_TS')

In [25]:
# Merge with second max order dataframe
max_Order=orders_head.join(right=max_order_ts,on='CUSTOMER_ID',how='inner')

#Filter for
filtered_df = max_Order.filter(F.col('ORDER_TS') == F.col('Max_Order_TS'))

In [26]:
filtered_df.show()

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"CUSTOMER_ID"  |"ORDER_ID"  |"TRUCK_ID"  |"LOCATION_ID"  |"DISCOUNT_ID"  |"SHIFT_ID"  |"SHIFT_START_TIME"  |"SHIFT_END_TIME"  |"ORDER_CHANNEL"  |"ORDER_TS"           |"SERVED_TS"  |"ORDER_CURRENCY"  |"ORDER_AMOUNT"  |"ORDER_TAX_AMOUNT"  |"ORDER_DISCOUNT_AMOUNT"  |"ORDER_TOTAL"  |"MAX_ORDER_TS"       |
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|193920         |129941626   |272         |7643.0         |NULL           |296484      |

In [27]:
filter_customer = filtered_df.join(CUSTOMERS, on='CUSTOMER_ID', how='left')

filter_customer.show()

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"CUSTOMER_ID"  |"ORDER_ID"  |"TRUCK_ID"  |"LOCATION_ID"  |"DISCOUNT_ID"  |"SHIFT_ID"  |"SHIFT_START_TIME"  |"SHIFT_END_TIME"  |"ORDER_CHANNEL"  |"ORDER_TS"           |"SERVED_TS"  |"ORDER_CURRENCY"  |"ORDER_AMOUNT"  |"ORDER_TAX_AMOUNT"  |"ORDER_DISCOUNT_AMOUNT"  |"ORDER_TOTAL"  |"MAX_ORDER_TS"       |"FIRST_NAME"  |"LAST_NAME"  |"CITY"     |"COUNTRY"      |"POSTAL_CODE"  |"PREFERRED_LANGUAGE"  |"GENDER"  |"FAVOURITE_BRAND"  |"MARITAL_ST

In [28]:
filter_customer_USA = filter_customer.where(filter_customer.COUNTRY == 'United States')

filter_customer_USA.show()

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"CUSTOMER_ID"  |"ORDER_ID"  |"TRUCK_ID"  |"LOCATION_ID"  |"DISCOUNT_ID"  |"SHIFT_ID"  |"SHIFT_START_TIME"  |"SHIFT_END_TIME"  |"ORDER_CHANNEL"  |"ORDER_TS"           |"SERVED_TS"  |"ORDER_CURRENCY"  |"ORDER_AMOUNT"  |"ORDER_TAX_AMOUNT"  |"ORDER_DISCOUNT_AMOUNT"  |"ORDER_TOTAL"  |"MAX_ORDER_TS"       |"FIRST_NAME"  |"LAST_NAME"  |"CITY"         |"COUNTRY"      |"POSTAL_CODE"  |"PREFERRED_LANGUAGE"  |"GENDER"     |"FAVOURITE_BRAND"  |"MARIT

In [31]:
order_with_detail = filter_customer_USA.join(right=orders_detail,on='ORDER_ID',how='inner')

order_with_detail.show()

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"ORDER_ID"  |"CUSTOMER_ID"  |"TRUCK_ID"  |"LOCATION_ID"  |"l_2jge_DISCOUNT_ID"  |"SHIFT_ID"  |"SHIFT_START_TIME"  |"SHIFT_END_TIME"  |"ORDER_CHANNEL"  |"ORDER_TS"           |"SERVED_TS"  |"ORDER_CURRENCY"  |"ORDER_AMOUNT"  |"ORDER_TAX_AMOUNT"  |"ORDER_DISCOUNT_AMOUNT"  |"ORDER_TOTAL"  |"MAX_O

In [32]:
order_with_detail = remove_dup_join_col(order_with_detail)
order_with_detail.show()

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"ORDER_ID"  |"CUSTOMER_ID"  |"TRUCK_ID"  |"LOCATION_ID"  |"SHIFT_ID"  |"SHIFT_START_TIME"  |"SHIFT_END_TIME"  |"ORDER_CHANNEL"  |"ORDER_TS"           |"SERVED_TS"  |"ORDER_CURRENCY"  |"ORDER_AMOUNT"  |"ORDER_TAX_AMOUNT"  |"ORDER_DISCOUNT_AMOUNT"  |"ORDER_TOTAL"  |"MAX_ORDER_TS"       |"FIRST_NAME"  |"LAST_NAME"  |"CITY"  

In [34]:
order_with_detail.count()

125999

In [35]:
filter_customer_USA.count()

46300

In [33]:
order_with_detail.write.save_as_table(table_name="ORDER_DETAILS_USA_MATCHED", mode='overwrite')
filter_customer_USA.write.save_as_table(table_name="ORDER_HEAD_USA_MATCHED", mode='overwrite')